In [2]:
library(Biobase) ; library(SingleCellExperiment)
library(Seurat)  ; library(scater)  ; library(scran)

### Functions 

In [3]:
# METRICS FUNCTIONS
num <- function(x){ return(as.numeric(as.character(x)))}
flatten <- function(x){return(as.vector(as.matrix(x)))}

### Load data from a pair of scRNA-seq datasets, filter genes and cells, integrate datasets and transfer cell-type labels

All steps here are posted for reproducibility and may take a long time to run. You can skip this step and go directly to the next chapter where we load already processed single-cell datasets with cell-type labels transfered between datasets. The raw single-cell datasets with all cells and all genes are provided as ExpressionSet objects and only for the two AML datasets, the other datasets are available on request. 

### Pre-processing pipeline:

__Filtering genes and cells:__
We load the data stored as `ExpressionSet` object, containing all the cells we consider and all the genes in a raw counts matrix, including the cell types labels fromt the original publications. Cell type labels are stored in `cellType_original` and sample name in `SubjectName`. We filter genes using the scran-implemented pipeline to keep genes with positive "biological" variance. As an additional step, we limit the number of cells to 30k in single-cell datasets by downsampling them with a set.seed(1). 

__Integrate datasets:__
We then use Seurat to integrate the two single-cell dataset with `Seurat::FindTransferAnchors()` using default parameters, and using found anchors as input for `Seurat::TransferData()`, from which we can obtain transfered cell-type labels from one dataset to the other. The transfered cell-type labels are then used to construct pseudo-bulks and perform cross-dataset pseudo-bulk deconvolution.

We then save the resulting tables which are used as input for the next steps.

In [4]:
### LOAD ALL FUNCTIONS ###
gene_intersection <- function(dataset1, dataset2){
    message('Finding gene intersect between two datasets')
    gene_intersect = intersect(rownames(dataset1), rownames(dataset2))
    message(paste0(length(gene_intersect), ' genes in common between two datasets. Subsetting datasets.'))
    return(gene_intersect)
}
downsample_cells <- function(sc, max.cells=30000){
    if( dim(sc)[2] > max.cells ){
        message(paste0('More than ',max.cells,' cells: downsampling to ',max.cells,' cells with seed(1)'))
        set.seed(1)
        sc <- sc[,sample(colnames(sc), max.cells)]
        sc$cellType_original = factor(sc$cellType_original)
        sc$SubjectName = factor(sc$SubjectName)
    } 
    return(sc)
}

select_variable_genes <- function(sc, cut.varbio=0.0){
    sce <- SingleCellExperiment(assays = list(counts = exprs(sc)))
    clusters <- quickCluster(sce, subset.row=NULL, assay.type="counts")
    sce <- computeSumFactors(sce, clusters=clusters)
    sce <- scater::logNormCounts(sce, size.factors = sizeFactors(sce))
    fit.genevar = modelGeneVar(sce)
    return(rownames(sc)[fit.genevar$bio > cut.varbio])
}

transfer_data <- function(dataset1, dataset2){
    message('Convert to Seurat objects and normalize both datasets')
    dataset1.srt <- CreateSeuratObject(counts = exprs(dataset1), project = "sc1", min.cells = 3, min.features = 200)
    Idents(object = dataset1.srt) <- dataset1$cellType_original
    dataset1.srt <- NormalizeData(dataset1.srt)
    dataset1.srt <- FindVariableFeatures(dataset1.srt, selection.method = "vst", nfeatures = 2000)
    dataset1.srt <- ScaleData(dataset1.srt)
    dataset2.srt <- CreateSeuratObject(counts = exprs(dataset2), project = "sc1", min.cells = 3, min.features = 200)
    dataset2.srt <- NormalizeData(dataset2.srt)
    dataset2.srt <- FindVariableFeatures(dataset2.srt, selection.method = "vst", nfeatures = 2000)
    dataset2.srt <- ScaleData(dataset2.srt)
    rm(dataset1) ; rm(dataset2) ; gc()
    message('Data integration: transfer cell-type labels from dataset1 to dataset2')
    anchors_1_to_2 <- FindTransferAnchors(reference = dataset1.srt, query = dataset2.srt)
    predictions_1_to_2 <- TransferData(anchorset = anchors_1_to_2, refdata = as.character(Idents(dataset1.srt)))
    return(predictions_1_to_2)
}

generate_final_datasets <- function(ref_sc, target_sc, var.genes){
    # Generate pseudo-bulk from single-cell ExpressionSet object
    # Use 'SubjectName' field to create pseudo-bulk samples 
    # Use 'cellType' field to create ground truth proportions in pData
    ref_sc = ref_sc[var.genes,]
    gene_intersect <- gene_intersection(ref_sc, target_sc)
    target_sc = target_sc[gene_intersect,]
    
    message('Setting cell-type factors as in reference')
    ref_sc$cellType_original = factor(ref_sc$cellType_original)
    target_sc$cellType_transfered = factor(target_sc$cellType_transfered,levels = levels(ref_sc$cellType_original))
    
    message('Build pseudo-bulks and iterate over samples to compute ground truth')
    p.bulks = aggregate(t(exprs(target_sc)), list(target_sc$SubjectName), sum)
    rownames(p.bulks) = p.bulks$`Group.1`
    p.bulks = p.bulks[,-1]
    list_ct_props= list() ; list_patients = rownames(p.bulks)
    for(n in 1:length(list_patients)){
        this_patient = list_patients[n]
        sel.patient = target_sc$SubjectName == this_patient
        sc.sub = target_sc[,sel.patient]
        celltype.sub = sc.sub$cellType_transfered
        list_ct_props[[n]] = table(celltype.sub) / sum(table(celltype.sub))
    }
    true.prop = do.call(rbind, list_ct_props)
    true.prop = true.prop[,colnames(true.prop) %in% levels(ref_sc$cellType_original)]
    true.prop = true.prop[,levels(ref_sc$cellType_original)]
    stopifnot(colnames(true.prop) == levels(ref_sc$cellType_original))
    rownames(true.prop) = list_patients
    message('Generate ExpressionSet objects containing pseudo-bulks in target dataset')
    p.bulks.es = ExpressionSet(t(p.bulks), phenoData = AnnotatedDataFrame(data.frame(true.prop)))
    return( list( ref_sc, p.bulks.es ) )
}

preprocess_data <- function(sc.1, sc.2, max.cells=30000){
    sc.1$cellType_original = factor(sc.1$cellType_original)
    sc.2$cellType_original = factor(sc.2$cellType_original)
    
    message(paste0('## DOWN-SAMPLING CELLS IF >',max.cells ,'K CELLS ##'))
    sc.1 = downsample_cells(sc.1, max.cells)
    sc.2 = downsample_cells(sc.2, max.cells)
    gc()

    message('## GENE INTERESECT BETWEEN TWO DATASETS ##')
    gene_intersect <- gene_intersection(sc.1, sc.2)
    sc.1 = sc.1[gene_intersect,]
    sc.2  = sc.2[gene_intersect,]
    stopifnot(all(rownames(sc.1) == rownames(sc.2)))
    gc()
    
    message('## FIND GENES WITH BIOLOGICAL VARIANCE > TECHNICAL VARIANCE ##')
    # Warning: This step takes long time to compute with large scRNA-seq data
    var.genes.1 = select_variable_genes(sc.1)
    var.genes.2 = select_variable_genes(sc.2)
    gc()
    
    message('## DATA INTEGRATION AND CELL-TYPE LABEL TRANSFER ##')
    transfered_labels_1to2 = transfer_data(sc.1, sc.2)
    intersect_labels_1to2 = intersect(colnames(sc.2), rownames(transfered_labels_1to2))
    stopifnot(unique(transfered_labels_1to2$predicted.id) > 1)
    sc.2 = sc.2[,intersect_labels_1to2]
    gc()
    transfered_labels_1to2 = transfered_labels_1to2[intersect_labels_1to2, ]
    stopifnot(all(rownames(transfered_labels_1to2) == colnames(sc.2)))
    #transfered_labels_2to1 = transfer_data(sc.2, sc.1)
    #intersect_labels_2to1 = intersect(colnames(sc.1), rownames(transfered_labels_2to1))
    #sc.1 = sc.1[,intersect_labels_2to1]
    #gc()
    #transfered_labels_2to1 = transfered_labels_2to1[intersect_labels_2to1, ]
    #stopifnot(all(rownames(transfered_labels_2to1) == colnames(sc.1)))
    sc.2$cellType_transfered = factor(transfered_labels_1to2$predicted.id, levels = levels(sc.1$cellType_original))
    sc.1$cellType_transfered = sc.1$cellType_original#factor(transfered_labels_2to1$predicted.id, levels = levels(sc.2$cellType_original))
    gc()
    
    message('## GENERATE FILTERED SC DATASET AND PSEUDO-BULKS ##')
    data_1to2 = generate_final_datasets(target_sc = sc.2, ref_sc = sc.1, var.genes.1)
    sc.1_ = data_1to2[[1]]
    sc.1_$cellType = factor(sc.1_$cellType_original)
    pbulks.2.es = data_1to2[[2]]
    stopifnot(all(rownames(sc.1_) == rownames(pbulks.2.es)))
    sc.2$cellType_original = sc.2$cellType_transfered ### to rm
    data_2to1 = generate_final_datasets(target_sc = sc.1, ref_sc = sc.2, var.genes.2)
    sc.2_ = data_2to1[[1]]
    sc.2_$cellType = factor(sc.2_$cellType_original)
    pbulks.1.es = data_2to1[[2]]
    stopifnot(all(rownames(sc.2_) == rownames(pbulks.1.es)))
    
    message('## RETURNING LIST WITH [SC1, PBULK1, SC2, PBULK2], PBULK1 WITH CELL-TYPE FROM SC2 AND VICE-VERSA ##')
    return( list(sc.1_, pbulks.1.es, sc.2_, pbulks.2.es) ) # 
}

We first load the two `ExpressionSet` objects from the full single-cell RNA-seq datasets containing raw counts, cell-type labels, and sample labels, for all cells and containing all unfiltered genes. 

In [5]:
### Load ExpressionSet objects from full raw single-cell RNA-seq datasets containing raw counts, cell-type
sc.dataset.1 = readRDS('data/pseudobulks_climb/raw/CRC_lee_sc_es.RDS')
sc.dataset.2  = readRDS('data/pseudobulks_climb/raw/CRC_khaliq_allGenes_sc_es.RDS')

In [7]:
sc.dataset.1$cellType_original = sc.dataset.1$cellType2

Here we launch our pipeline to filter cells/genes, integrate both datasets, and obtain reference single-cell dataset from dataset1 with pseudo-bulks made from dataset2, and similarly, reference single-cell from dataset2 with associated pseudo-bulk from dataset1. Thanks to data integration, cell-type labels match between reference and pseudo-bulks.

In [8]:
preproc_dat = preprocess_data(sc.dataset.1, sc.dataset.2)

## DOWN-SAMPLING CELLS IF >30000K CELLS ##

More than 30000 cells: downsampling to 30000 cells with seed(1)

More than 30000 cells: downsampling to 30000 cells with seed(1)

## GENE INTERESECT BETWEEN TWO DATASETS ##

Finding gene intersect between two datasets

11578 genes in common between two datasets. Subsetting datasets.

## FIND GENES WITH BIOLOGICAL VARIANCE > TECHNICAL VARIANCE ##

Warning message in (function (x, sizes, min.mean = NULL, positive = FALSE, scaling = NULL) :
“encountered non-positive size factor estimates”
## DATA INTEGRATION AND CELL-TYPE LABEL TRANSFER ##

Convert to Seurat objects and normalize both datasets

Centering and scaling data matrix

Centering and scaling data matrix

Data integration: transfer cell-type labels from dataset1 to dataset2

Performing PCA on the provided reference using 1999 features as input.

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 30137 anchors

Filtering anchors

	Retained 10129 anchors

Finding in

In [9]:
# We extract the final dataset from the run above
sc.dataset.1 = preproc_dat[[1]]
pbulks.1.es = preproc_dat[[2]]
sc.dataset.2 = preproc_dat[[3]]
pbulks.2.es = preproc_dat[[4]]
gc()

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,8299152,443.3,15047914,803.7,NA,15047914,803.7
Vcells,326089497,2487.9,2688527323,20511.9,102400,3360656645,25639.8


Save the processed pair of single-cell datasets and pseudo-bulks into `data/pseudobulks_climb` folder

In [16]:
saveRDS(sc.dataset.1, 'data/pseudobulks_climb/CRC_lee_sc_es.RDS')
saveRDS(pbulks.2.es, 'data/pseudobulks_climb/CRC_khaliq_pbulks_lee_labs_es.RDS')

In [17]:
saveRDS(sc.dataset.2, 'data/pseudobulks_climb/CRC_khaliq_sc_es.RDS')
saveRDS(pbulks.1.es, 'data/pseudobulks_climb/CRC_lee_pbulks_khaliq_labs_es.RDS')

We thus obtained pairs of single-cell reference and pseudo-bulk samples. Here is a description of these file for Van Galen and Naldini pairs of datasets:
- `sc.dataset.1`: contains single-cell datasets from Van Galen datasets. Gene were filtered using scran::modelGeneVar and using only positive biological variance (namely, gene for which the estimated biological variance is higher than the estimated technical variance). No filters was applied on cells.
- `pbulks.2.es`: contains the pseudo-bulk made from Naldini single-cell dataset. The cell-type were transfered from Van Galen to Naldini single-cells. Then, the Naldini dataset is assembled into pseudo-bulks using its original samples (corresponding to patient at day 0 pre-treatment), and ground truth proportions of cell-types using transfered labels from Van Galen were recorded in the phenoData compartment of the ExpressionSet object.
- `sc.dataset.2`: contains single-cell datasets from Naldini datasets. Gene were filtered using scran::modelGeneVar and using only positive biological variance (namely, gene for which the estimated biological variance is higher than the estimated technical variance). Cells were sub-sampled to keep only 30'000 cells (using random sample function with set.seed(1) for reproducibility). Sub-sampling is done to obtain a single-cell matrix <1.0Gb to be able to use CiberSortX method. 
- `pbulks.2.es`: contains the pseudo-bulk made from Van Galen single-cell dataset. The cell-type were transfered from Naldini to Van Galen single-cells. Then, the Van Galen dataset is assembled into pseudo-bulks using its original samples (corresponding to patient at day 0 pre-treatment), and ground truth proportions of cell-types using transfered labels from Van Galen were recorded in the phenoData compartment of the ExpressionSet object.

The deconvolution of the datasets pre-processed is done in the following notebook: `Fig3_deconvolution_panel.ipynb` 

### One peculiar case: pairs of datasets in Glioblastoma 

In the case of Glioblastoma, both single-cell comes from the same datasets (Neftel et al.) but one was sequenced with 10X and the other one with SmartSeq2. We thus performed 10x > SmartSeq2, and SmartSeq2 > 10X cross-dataset analysis. Another peculiarity of this dataset is that only the SmartSeq2 data had available cell-type labels. We thus use these labels in both 10x > SmartSeq2, and SmartSeq2 > 10X analysis. 

### CRC cross-dataset analysis

In the colorectal cancer analysis, Khaliq et al. dataset did not have available cell type labels except from a CMS subtype probability per single cell. We thus used Lee et al. cell-type labels for both datasets.